In [ ]:
#Importing all libraries
import requests # to make TMDB API calls
import locale # to format currency as USD
locale.setlocale( locale.LC_ALL, '' )
import pandas as pd

In [ ]:
#Initiate the API Key
api_key="2fd41becbab07c05e0b4f5840d4ef105" 

##Get Movie data 

In [ ]:
import ast
movie_data= pd.DataFrame([])
#Storing Movies and Revenue data in a dataframe
for i in range(1,6):
    response = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&primary_release_date.gte=2021-01-01&primary_release_date.lte=2021-12-31&sort_by=revenue.desc&page='+str(i))
    highest_revenue_ever = response.json()
    highest_revenue_films_ever = highest_revenue_ever['results']
    movie_data = movie_data.append(highest_revenue_films_ever,ignore_index=True,sort=False)
movie_data=movie_data.astype(str)
movie_data['genre_ids'] = movie_data['genre_ids'].apply(lambda x: ast.literal_eval(x))
movie_data = movie_data.explode('genre_ids')
movie_data=movie_data.astype(str)

In [ ]:
movie_data.head(3)

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/iQFcwSGbZXMkeyKrxbPnwnRo5fl.jpg,28,634649,en,Spider-Man: No Way Home,Peter Parker is unmasked and no longer able to...,2506.159,/1g0dhYtq4irTY1GPXvft6k4YLjm.jpg,2021-12-15,Spider-Man: No Way Home,False,8.1,13772
0,False,/iQFcwSGbZXMkeyKrxbPnwnRo5fl.jpg,12,634649,en,Spider-Man: No Way Home,Peter Parker is unmasked and no longer able to...,2506.159,/1g0dhYtq4irTY1GPXvft6k4YLjm.jpg,2021-12-15,Spider-Man: No Way Home,False,8.1,13772
0,False,/iQFcwSGbZXMkeyKrxbPnwnRo5fl.jpg,878,634649,en,Spider-Man: No Way Home,Peter Parker is unmasked and no longer able to...,2506.159,/1g0dhYtq4irTY1GPXvft6k4YLjm.jpg,2021-12-15,Spider-Man: No Way Home,False,8.1,13772


##Getting Revenue Data

In [ ]:
revenue_data= pd.DataFrame([])
for i in range(1,6):
    response = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&primary_release_date.gte=2021-01-01&primary_release_date.lte=2021-12-31&sort_by=revenue.desc&page='+str(i))
    highest_revenue_ever = response.json()
    highest_revenue_films_ever = highest_revenue_ever['results']
    for film in highest_revenue_films_ever:
    # print(film['title'])

      film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key+'&language=en-US')
      film_revenue = film_revenue.json()
    # print(film_revenue)

    # print(locale.currency(film_revenue['revenue'], grouping=True ))
      revenue_data= revenue_data.append(film_revenue,ignore_index=True,sort=False)
revenue_data=revenue_data.astype(str)

##Getting Genre data

In [ ]:
response=requests.get('https://api.themoviedb.org/3/genre/movie/list?api_key=' + api_key )
genre_response= response.json()
genre_data=pd.json_normalize(genre_response, 'genres')
genre_data=genre_data.astype(str)

##Connecting SQLite3

In [ ]:
import sqlite3
conn = sqlite3.connect('TMDB.db')

In [ ]:
##push the dataframe to sql 
movie_data.to_sql("movie_data", conn, if_exists="replace")

##create the table to push movie_data 

conn.execute(
    """
    create table Movies as 
    select * from movie_data
    """)

In [ ]:
##push the dataframe to sql 
revenue_data.to_sql("revenue_data", conn, if_exists="replace")

##create the table to push revenue_data 

conn.execute(
    """
    create table Movies_revenue as 
    select * from revenue_data
    """)

In [ ]:
##push the dataframe to sql 
genre_data.to_sql("genre_data", conn, if_exists="replace")

##create the table to push genre_data 

conn.execute(
    """
    create table Movies_genres as 
    select * from genre_data
    """)